# Final Exercise - Putting it All Together

In this last exercise, you'll write **a full ETL pipeline for the GDP data**. That means you'll extract the World Bank data, transform the data, and load the data all in one go. In other words, you'll want **one Python script that can do the entire process.**

Why would you want to do this? Imagine working for a company that creates new data every day. **As new data comes in, you'll want to write software that periodically and automatically extracts, transforms, and loads the data.**

To give you a sense for what this is like, you'll extract the GDP data one line at a time. You'll then transform that line of data and load the results into a SQLite database. The code in this exercise is somewhat tricky.

Here is an explanation of how this Jupyter notebook is organized:
1. The first cell connects to a SQLite database called worldbank.db and creates a table to hold the gdp data. You do not need to do anything in this code cell other than executing the cell.
2. The second cell has a function called extract_line(). You don't need to do anything in this code cell either besides executing the cell. This function is a [Python generator](https://wiki.python.org/moin/Generators). You don't need to understand how this works in order to complete the exercise. 
    1. Essentially, a generator is like a regular function except **instead of a return** statement, a generator has a **yield statement.**
    2. **Generators allow you to use functions in a for loop**. In essence, this function will allow you to **read in a data file one line at a time**, run a transformation on that row of data, and then move on to the next row in the file.
3. The third cell contains a function called transform_indicator_data(). This function receives a line from the csv file and transforms the data in preparation for a load step.
4. The fourth cell contains a function called load_indicator_data(), which loads the trasnformed data into the gdp table in the worldbank.db database.
5. The fifth cell runs the ETL pipeilne
6. The sixth cell runs a query against the database to make sure everything worked correctly.

You'll need to **modify the third and fourth cells.**

In [1]:
# run this cell to create a database and a table, called gdp, to hold the gdp data
# You do not need to change anything in this code cell

import sqlite3

# connect to the database
# the database file will be worldbank.db
# note that sqlite3 will create this database file if it does not exist already
conn = sqlite3.connect('worldbank.db')

# get a cursor
cur = conn.cursor()

# drop the test table in case it already exists
cur.execute("DROP TABLE IF EXISTS gdp")

# create the test table including project_id as a primary key
cur.execute("CREATE TABLE gdp (countryname TEXT, countrycode TEXT, year INTEGER, gdp REAL, PRIMARY KEY (countrycode, year));")

conn.commit()
conn.close()

In [2]:
# Generator for reading in one line at a time

### generators are useful for data sets that are too large to fit in RAM

# You do not need to change anything in this code cell

def extract_lines(file):
    while True:
        line = file.readline()
        if not line:
            break
        yield line

In [3]:
# these are "countryname" values that are not actually countries
non_countries = ['World',
 'High income',
 'OECD members',
 'Post-demographic dividend',
 'IDA & IBRD total',
 'Low & middle income',
 'Middle income',
 'IBRD only',
 'East Asia & Pacific',
 'Europe & Central Asia',
 'North America',
 'Upper middle income',
 'Late-demographic dividend',
 'European Union',
 'East Asia & Pacific (excluding high income)',
 'East Asia & Pacific (IDA & IBRD countries)',
 'Euro area',
 'Early-demographic dividend',
 'Lower middle income',
 'Latin America & Caribbean',
 'Latin America & the Caribbean (IDA & IBRD countries)',
 'Latin America & Caribbean (excluding high income)',
 'Europe & Central Asia (IDA & IBRD countries)',
 'Middle East & North Africa',
 'Europe & Central Asia (excluding high income)',
 'South Asia (IDA & IBRD)',
 'South Asia',
 'Arab World',
 'IDA total',
 'Sub-Saharan Africa',
 'Sub-Saharan Africa (IDA & IBRD countries)',
 'Sub-Saharan Africa (excluding high income)',
 'Middle East & North Africa (excluding high income)',
 'Middle East & North Africa (IDA & IBRD countries)',
 'Central Europe and the Baltics',
 'Pre-demographic dividend',
 'IDA only',
 'Least developed countries: UN classification',
 'IDA blend',
 'Fragile and conflict affected situations',
 'Heavily indebted poor countries (HIPC)',
 'Low income',
 'Small states',
 'Other small states',
 'Not classified',
 'Caribbean small states',
 'Pacific island small states']

In [4]:
import pandas as pd
import numpy as np
import sqlite3

In [5]:
# test with one line

with open('../data/gdp_data.csv') as f:
    for i, line in enumerate(f):
        if i==6:
            data = line.split(',')
            break

In [6]:
data

['"Afghanistan"',
 '"AFG"',
 '"GDP (current US$)"',
 '"NY.GDP.MKTP.CD"',
 '"537777811.111111"',
 '"548888895.555556"',
 '"546666677.777778"',
 '"751111191.111111"',
 '"800000044.444444"',
 '"1006666637.77778"',
 '"1399999966.66667"',
 '"1673333417.77778"',
 '"1373333366.66667"',
 '"1408888922.22222"',
 '"1748886595.55556"',
 '"1831108971.11111"',
 '"1595555475.55556"',
 '"1733333264.44444"',
 '"2155555497.77778"',
 '"2366666615.55556"',
 '"2555555566.66667"',
 '"2953333417.77778"',
 '"3300000108.88889"',
 '"3697940409.61098"',
 '"3641723321.99546"',
 '"3478787909.09091"',
 '""',
 '""',
 '""',
 '""',
 '""',
 '""',
 '""',
 '""',
 '""',
 '""',
 '""',
 '""',
 '""',
 '""',
 '""',
 '""',
 '""',
 '""',
 '""',
 '"2461665937.89386"',
 '"4128820723.04713"',
 '"4583644246.48061"',
 '"5285465685.86423"',
 '"6275073571.54659"',
 '"7057598406.61553"',
 '"9843842455.48323"',
 '"10190529882.4878"',
 '"12486943505.7381"',
 '"15936800636.2487"',
 '"17930239399.8149"',
 '"20536542736.7297"',
 '"202642539

In [7]:
for i, datum in enumerate(data):
    data[i] = datum.replace('"','')

In [8]:
data

['Afghanistan',
 'AFG',
 'GDP (current US$)',
 'NY.GDP.MKTP.CD',
 '537777811.111111',
 '548888895.555556',
 '546666677.777778',
 '751111191.111111',
 '800000044.444444',
 '1006666637.77778',
 '1399999966.66667',
 '1673333417.77778',
 '1373333366.66667',
 '1408888922.22222',
 '1748886595.55556',
 '1831108971.11111',
 '1595555475.55556',
 '1733333264.44444',
 '2155555497.77778',
 '2366666615.55556',
 '2555555566.66667',
 '2953333417.77778',
 '3300000108.88889',
 '3697940409.61098',
 '3641723321.99546',
 '3478787909.09091',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '2461665937.89386',
 '4128820723.04713',
 '4583644246.48061',
 '5285465685.86423',
 '6275073571.54659',
 '7057598406.61553',
 '9843842455.48323',
 '10190529882.4878',
 '12486943505.7381',
 '15936800636.2487',
 '17930239399.8149',
 '20536542736.7297',
 '20264253973.7858',
 '20616104298.4975',
 '19215562179.0117',
 '19469022207.6852',
 '20815300220.0428',
 '\n']

In [9]:
country = data[0]
country

'Afghanistan'

In [10]:
len(data)

63

In [11]:
data_array = np.asarray(data)
data_array

array(['Afghanistan', 'AFG', 'GDP (current US$)', 'NY.GDP.MKTP.CD',
       '537777811.111111', '548888895.555556', '546666677.777778',
       '751111191.111111', '800000044.444444', '1006666637.77778',
       '1399999966.66667', '1673333417.77778', '1373333366.66667',
       '1408888922.22222', '1748886595.55556', '1831108971.11111',
       '1595555475.55556', '1733333264.44444', '2155555497.77778',
       '2366666615.55556', '2555555566.66667', '2953333417.77778',
       '3300000108.88889', '3697940409.61098', '3641723321.99546',
       '3478787909.09091', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '2461665937.89386', '4128820723.04713',
       '4583644246.48061', '5285465685.86423', '6275073571.54659',
       '7057598406.61553', '9843842455.48323', '10190529882.4878',
       '12486943505.7381', '15936800636.2487', '17930239399.8149',
       '20536542736.7297', '20264253973.7858', '20616104298.4975',
       '19215562179.0117', '19469022207.6852'

In [12]:
data_array = data_array.reshape(1, 63)

In [13]:
data_array

array([['Afghanistan', 'AFG', 'GDP (current US$)', 'NY.GDP.MKTP.CD',
        '537777811.111111', '548888895.555556', '546666677.777778',
        '751111191.111111', '800000044.444444', '1006666637.77778',
        '1399999966.66667', '1673333417.77778', '1373333366.66667',
        '1408888922.22222', '1748886595.55556', '1831108971.11111',
        '1595555475.55556', '1733333264.44444', '2155555497.77778',
        '2366666615.55556', '2555555566.66667', '2953333417.77778',
        '3300000108.88889', '3697940409.61098', '3641723321.99546',
        '3478787909.09091', '', '', '', '', '', '', '', '', '', '', '',
        '', '', '', '', '', '', '', '', '2461665937.89386',
        '4128820723.04713', '4583644246.48061', '5285465685.86423',
        '6275073571.54659', '7057598406.61553', '9843842455.48323',
        '10190529882.4878', '12486943505.7381', '15936800636.2487',
        '17930239399.8149', '20536542736.7297', '20264253973.7858',
        '20616104298.4975', '19215562179.0117', '19

In [14]:
data_array.shape

(1, 63)

In [15]:
with open('../data/gdp_data.csv') as f:
    for i, line in enumerate(f):
        if i==4:
            data4 = line.split(',')
            break
            
colnames = []
# get rid of quote marks in the results to make the data easier to work with

for i, datum in enumerate(data4):
    colnames.append(datum.replace('"',''))

colnames

['Country Name',
 'Country Code',
 'Indicator Name',
 'Indicator Code',
 '1960',
 '1961',
 '1962',
 '1963',
 '1964',
 '1965',
 '1966',
 '1967',
 '1968',
 '1969',
 '1970',
 '1971',
 '1972',
 '1973',
 '1974',
 '1975',
 '1976',
 '1977',
 '1978',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '\n']

In [16]:
df = pd.DataFrame(data_array, columns=colnames).replace('', np.nan)

df.drop(columns=['Indicator Name', 'Indicator Code', '\n'], axis=1, inplace=True)

df_melt = df.melt(id_vars=['Country Name', 'Country Code'], var_name='year', value_name='gdp')

In [17]:
df

,Country Name,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Afghanistan,AFG,537777811.111111,548888895.555556,546666677.777778,751111191.111111,800000044.444444,1006666637.77778,1399999966.66667,1673333417.77778,...,10190529882.4878,12486943505.7381,15936800636.2487,17930239399.8149,20536542736.7297,20264253973.7858,20616104298.4975,19215562179.0117,19469022207.6852,20815300220.0428


In [18]:
df_melt

,Country Name,Country Code,year,gdp
0,Afghanistan,AFG,1960,537777811.111111
1,Afghanistan,AFG,1961,548888895.555556
2,Afghanistan,AFG,1962,546666677.777778
3,Afghanistan,AFG,1963,751111191.111111
4,Afghanistan,AFG,1964,800000044.444444
5,Afghanistan,AFG,1965,1006666637.77778
6,Afghanistan,AFG,1966,1399999966.66667
7,Afghanistan,AFG,1967,1673333417.77778
8,Afghanistan,AFG,1968,1373333366.66667
9,Afghanistan,AFG,1969,1408888922.22222


In [26]:
df_melt.head(1).gdp

0    537777811.111111
Name: gdp, dtype: object

In [19]:
# TODO: Iterate through the rows in df_melt
# For each row, extract the country, countrycode, year, and gdp values into a list like this:
#     [country, countrycode, year, gdp]
# If the gdp value is not null, append the row (in the form of a list) to the results variable
# Finally, return the results list after iterating through the df_melt data
# HINT: the iterrows() method would be useful
# HINT: to check if gdp is equal to nan, you might want to convert gdp to a string and compare to the string 'nan'
results = []

for i, row in df_melt.iterrows():
    if row[3] != 'nan':
        results.append(list(row))
results


[['Afghanistan', 'AFG', '1960', '537777811.111111'],
 ['Afghanistan', 'AFG', '1961', '548888895.555556'],
 ['Afghanistan', 'AFG', '1962', '546666677.777778'],
 ['Afghanistan', 'AFG', '1963', '751111191.111111'],
 ['Afghanistan', 'AFG', '1964', '800000044.444444'],
 ['Afghanistan', 'AFG', '1965', '1006666637.77778'],
 ['Afghanistan', 'AFG', '1966', '1399999966.66667'],
 ['Afghanistan', 'AFG', '1967', '1673333417.77778'],
 ['Afghanistan', 'AFG', '1968', '1373333366.66667'],
 ['Afghanistan', 'AFG', '1969', '1408888922.22222'],
 ['Afghanistan', 'AFG', '1970', '1748886595.55556'],
 ['Afghanistan', 'AFG', '1971', '1831108971.11111'],
 ['Afghanistan', 'AFG', '1972', '1595555475.55556'],
 ['Afghanistan', 'AFG', '1973', '1733333264.44444'],
 ['Afghanistan', 'AFG', '1974', '2155555497.77778'],
 ['Afghanistan', 'AFG', '1975', '2366666615.55556'],
 ['Afghanistan', 'AFG', '1976', '2555555566.66667'],
 ['Afghanistan', 'AFG', '1977', '2953333417.77778'],
 ['Afghanistan', 'AFG', '1978', '3300000108.88

In [28]:
for i, row in df_melt.iterrows():
    if row[3] != 'nan':
        print(list(row))

['Afghanistan', 'AFG', '1960', '537777811.111111']
['Afghanistan', 'AFG', '1961', '548888895.555556']
['Afghanistan', 'AFG', '1962', '546666677.777778']
['Afghanistan', 'AFG', '1963', '751111191.111111']
['Afghanistan', 'AFG', '1964', '800000044.444444']
['Afghanistan', 'AFG', '1965', '1006666637.77778']
['Afghanistan', 'AFG', '1966', '1399999966.66667']
['Afghanistan', 'AFG', '1967', '1673333417.77778']
['Afghanistan', 'AFG', '1968', '1373333366.66667']
['Afghanistan', 'AFG', '1969', '1408888922.22222']
['Afghanistan', 'AFG', '1970', '1748886595.55556']
['Afghanistan', 'AFG', '1971', '1831108971.11111']
['Afghanistan', 'AFG', '1972', '1595555475.55556']
['Afghanistan', 'AFG', '1973', '1733333264.44444']
['Afghanistan', 'AFG', '1974', '2155555497.77778']
['Afghanistan', 'AFG', '1975', '2366666615.55556']
['Afghanistan', 'AFG', '1976', '2555555566.66667']
['Afghanistan', 'AFG', '1977', '2953333417.77778']
['Afghanistan', 'AFG', '1978', '3300000108.88889']
['Afghanistan', 'AFG', '1979', 

In [20]:
type(results[0][3])

str